In [3]:
import pandas as pd
import numpy as np
from datetime import date
total = 22606

# FUNCTIONS
#### String to date time:
Transform files to desirable format
#### Join times:
Combine the overlaped intervals from the two different sources
#### Total alert time:
Computes the total amount of minutes pertaining to the time intervals
#### Roc values:
Returns list with ROC parameteres + Youden

In [4]:
def strings_to_datetime(df):
    df['hora inici'] = (pd.to_datetime(df['dia'] + ' ' + df['hora inici']))
    df['hora final'] = (pd.to_datetime(df['dia'] + ' ' + df['hora final']))
    df['dia'] = (pd.to_datetime(df['dia'])).dt.date

    return df

def join_times(x):
    startdf = pd.DataFrame({'time':x['hora inici'], 'what':1})
    enddf = pd.DataFrame({'time':x['hora final'], 'what':-1})
    mergdf = pd.concat([startdf, enddf]).sort_values('time')
    mergdf['running'] = mergdf['what'].cumsum()
    mergdf['newwin'] = mergdf['running'].eq(1) & mergdf['what'].eq(1)
    mergdf['group'] = mergdf['newwin'].cumsum()
    x['group'] = mergdf['group'].loc[mergdf['what'].eq(1)]
    return mergdf

def total_alert_time(x):
    total_time = ((x['hora final'] - x['hora inici'])/np.timedelta64(1,'m')).sum()
    return total_time

def roc_values_min(x):
    true_x = x[x['resultat'] == 'VERDADERO']
    false_x = x[x['resultat'] == 'FALSO']
    true_positives = len(true_x) #events
    false_positives_min = total_alert_time(false_x) #minutes
    false_positives_event = len(false_x) #events
    true_negatives = 22606 - total_alert_time(x) #minutes
    false_negatives = 33 - true_positives #events
    TPR = true_positives/(true_positives + false_negatives)
    FPR = false_positives_min/(false_positives_min + true_negatives)
    TNR = true_negatives/(true_negatives + false_positives_min)
    youden = TPR + TNR - 1
    
    return [true_positives, false_positives_event, false_positives_min, TPR, FPR, youden]

def roc_values_event_16(x):
    true_x = x[x['resultat'] == 'VERDADERO']
    false_x = x[x['resultat'] == 'FALSO']
    true_positives = len(true_x)
    false_positives = len(false_x)
    true_negatives = (22606 - total_alert_time(x))/16
    false_negatives = 33 - true_positives #events
    TPR = true_positives/(true_positives + false_negatives)
    FPR = false_positives/(false_positives + true_negatives)
    TNR = true_negatives/(true_negatives + false_positives)
    youden = TPR + TNR - 1
    
    return [true_positives, false_positives, true_negatives, TPR, FPR, youden]

def roc_values_event_mean(x):
    true_x = x[x['resultat'] == 'VERDADERO']
    false_x = x[x['resultat'] == 'FALSO']
    mean_duration = total_alert_time(x)/len(x)
    true_positives = len(true_x)
    false_positives = len(false_x)
    true_negatives = (22606 - total_alert_time(x))/mean_duration
    false_negatives = 33 - true_positives #events
    TPR = true_positives/(true_positives + false_negatives)
    FPR = false_positives/(false_positives + true_negatives)
    TNR = true_negatives/(true_negatives + false_positives)
    youden = TPR + TNR - 1
    
    return [true_positives, false_positives, true_negatives, TPR, FPR, youden]

    

#### Read files
* Intervals1: data from the morphological approach
* Intervals2: data from the derivative approach

In [5]:
alba = pd.DataFrame(columns=['TP', 'FP events','FP minutes'])
deltas = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for delta in deltas:
    file2 = f'intervals_collections/intervals_alba/alba-{delta}.csv'
    Intervals2 = pd.read_csv(file2, index_col=None, delimiter=';')
    Intervals2['metode'] = 2
    strings_to_datetime(Intervals2)
    alba.loc[len(alba)] =roc_values_min(Intervals2)[0:3]
alba

,TP,FP events,FP minutes
0,33.0,446.0,7532.0
1,30.0,298.0,2981.0
2,28.0,162.0,1217.0
3,26.0,84.0,578.0
4,21.0,50.0,348.0
5,20.0,28.0,171.0
6,17.0,18.0,143.0
7,15.0,8.0,90.0
8,12.0,6.0,82.0


In [6]:
alba = pd.DataFrame(columns=['Cond', 'Minutes','Mean duration', '16 min duration'])
deltas = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
for delta in deltas:
    file2 = f'intervals_collections/intervals_alba/alba-{delta}.csv'
    Intervals2 = pd.read_csv(file2, index_col=None, delimiter=';')
    Intervals2['metode'] = 2
    strings_to_datetime(Intervals2)
    alba.loc[len(alba)] = [delta, roc_values_min(Intervals2)[5], roc_values_event_mean(Intervals2)[5], roc_values_event_16(Intervals2)[5]]
alba

,Cond,Minutes,Mean duration,16 min duration
0,0.2,0.642151,0.614929,0.654464
1,0.3,0.772679,0.756719,0.707399
2,0.4,0.793352,0.781757,0.737947
3,0.5,0.761839,0.754190,0.729350
4,0.6,0.620759,0.615846,0.601203
5,0.7,0.598410,0.594981,0.586262
6,0.8,0.508768,0.507221,0.502379
7,0.9,0.450533,0.450437,0.448849
8,1.0,0.359987,0.360401,0.359367


In [7]:
anna = pd.DataFrame(columns=['Cond', 'Minutes','Mean duration', '16 min duration'])
for corr in np.arange(0.977, 0.994, 0.001):
    file1 = f'intervals_collections/intervals_anna/intervals-{corr}.csv'
    Intervals1 = pd.read_csv(file1, index_col=0, delimiter=';')
    Intervals1['metode'] = 1
    strings_to_datetime(Intervals1)
    anna.loc[len(anna)] = [corr, roc_values_min(Intervals1)[5], roc_values_event_mean(Intervals1)[5], roc_values_event_16(Intervals1)[5]]
anna

,Cond,Minutes,Mean duration,16 min duration
0,0.977,0.603604,0.594837,0.720286
1,0.978,0.577112,0.569338,0.687632
2,0.979,0.613005,0.607135,0.707204
3,0.980,0.641786,0.635426,0.726226
4,0.981,0.656167,0.650858,0.724185
5,0.982,0.668745,0.664093,0.727211
6,0.983,0.682350,0.680974,0.726318
7,0.984,0.724599,0.723724,0.751711
8,0.985,0.775315,0.775297,0.781475
9,0.986,0.657969,0.660733,0.650852


In [8]:
anna = pd.DataFrame(columns=['TP', 'FP events','FP minutes'])
for corr in np.arange(0.977, 0.994, 0.001):
    file1 = f'intervals_collections/intervals_anna/intervals-{corr}.csv'
    Intervals1 = pd.read_csv(file1, index_col=0, delimiter=';')
    Intervals1['metode'] = 1
    strings_to_datetime(Intervals1)
    anna.loc[len(anna)] =roc_values_min(Intervals1)[0:3]
anna

,TP,FP events,FP minutes
0,33.0,313.0,8469.0
1,31.0,288.0,7780.0
2,31.0,275.0,7050.0
3,31.0,257.0,6431.0
4,30.0,230.0,5492.0
5,29.0,192.0,4572.0
6,28.0,159.0,3642.0
7,28.0,129.0,2724.0
8,28.0,92.0,1618.0
9,23.0,65.0,873.0


Join the two sets of data and create a DataFrame with merged overlapping intervals

# RESULTS
* ### Combined DataFrame

In [23]:
merged = pd.merge_ordered(Intervals1, Intervals2,
              on=['dia', 'hora inici', 'hora final', 'resultat', 'metode'])
mergdf = join_times(merged)
grouped_df = merged.groupby('group')
combined = pd.DataFrame()
combined = grouped_df.max()
combined['hora inici'] = grouped_df['hora inici'].min()
combined['metode'] = grouped_df['metode'].mean()
combined.to_csv('combined.csv')

* ### Amount of TP and FP for each method

In [24]:

print('Combined TP:', len(combined[combined['resultat'] == 'VERDADERO']))
print('Combined FP: ',len(combined[combined['resultat'] == 'FALSO']))
print('Anna TP: ', len(Intervals1[Intervals1['resultat'] == 'VERDADERO']))
print('Anna FP: ', len(Intervals1[Intervals1['resultat'] == 'FALSO']))
print('Alba TP: ', len(Intervals2[Intervals2['resultat'] == 'VERDADERO']))
print('Alba FP: ', len(Intervals2[Intervals2['resultat'] == 'FALSO']))
print(total_alert_time(combined))

Combined TP: 16
Combined FP:  7
Anna TP:  5
Anna FP:  1
Alba TP:  12
Alba FP:  6
256.5


In [25]:
combined[combined['resultat'] == 'VERDADERO']

,dia,hora inici,hora final,resultat,metode
group,,,,,
1,2001-04-15,2001-04-15 13:42:00,2001-04-15 14:06:00,VERDADERO,2.0
2,2001-04-26,2001-04-26 13:02:30,2001-04-26 13:10:00,VERDADERO,1.5
3,2001-12-13,2001-12-13 14:24:00,2001-12-13 14:29:00,VERDADERO,2.0
4,2004-07-16,2004-07-16 13:50:00,2004-07-16 14:01:00,VERDADERO,2.0
5,2004-07-20,2004-07-20 12:25:00,2004-07-20 12:31:00,VERDADERO,2.0
8,2004-08-15,2004-08-15 12:37:00,2004-08-15 12:39:00,VERDADERO,2.0
9,2005-09-09,2005-09-09 09:53:00,2005-09-09 10:05:00,VERDADERO,2.0
13,2006-12-05,2006-12-05 10:24:00,2006-12-05 10:55:00,VERDADERO,2.0
14,2011-08-09,2011-08-09 08:00:00,2011-08-09 08:14:00,VERDADERO,2.0


In [26]:
combined

,dia,hora inici,hora final,resultat,metode
group,,,,,
1,2001-04-15,2001-04-15 13:42:00,2001-04-15 14:06:00,VERDADERO,2.0
2,2001-04-26,2001-04-26 13:02:30,2001-04-26 13:10:00,VERDADERO,1.5
3,2001-12-13,2001-12-13 14:24:00,2001-12-13 14:29:00,VERDADERO,2.0
4,2004-07-16,2004-07-16 13:50:00,2004-07-16 14:01:00,VERDADERO,2.0
5,2004-07-20,2004-07-20 12:25:00,2004-07-20 12:31:00,VERDADERO,2.0
6,2004-08-14,2004-08-14 09:42:00,2004-08-14 09:47:00,FALSO,2.0
7,2004-08-14,2004-08-14 11:34:00,2004-08-14 11:35:00,FALSO,2.0
8,2004-08-15,2004-08-15 12:37:00,2004-08-15 12:39:00,VERDADERO,2.0
9,2005-09-09,2005-09-09 09:53:00,2005-09-09 10:05:00,VERDADERO,2.0


In [114]:
def margin(x):
    x['diff'] = ((x['hora inici'] - x['hora final'].shift(1))/np.timedelta64(1, 'm')) > 2
    x['group 2'] = x['diff'].cumsum()
    grouped_df = x.groupby('group 2')
    x = pd.DataFrame()
    x = grouped_df.max()
    x['hora inici'] = grouped_df['hora inici'].min()
    x['metode'] = grouped_df['metode'].mean()
    return x

In [115]:
file1 = 'intervals_collections/intervals_anna/intervals-0.985.csv'
Intervals1 = pd.read_csv(file1, index_col=0, delimiter=';')
Intervals1['metode'] = 1
strings_to_datetime(Intervals1)
file2 = 'intervals_collections/intervals_alba/alba-0.4.csv'
Intervals2 = pd.read_csv(file2, index_col=None, delimiter=';')
Intervals2['metode'] = 2
strings_to_datetime(Intervals2)
merged = pd.merge_ordered(Intervals1, Intervals2,
              on=['dia', 'hora inici', 'hora final', 'resultat', 'metode'])
mergdf = join_times(merged)
grouped_df = merged.groupby('group')
combined = pd.DataFrame()
combined = grouped_df.max()
combined['hora inici'] = grouped_df['hora inici'].min()
combined['metode'] = grouped_df['metode'].mean()
combined.to_csv('combined.csv')

combined

,dia,hora inici,hora final,resultat,metode
group,,,,,
1,2001-04-15,2001-04-15 07:22:00,2001-04-15 07:26:00,FALSO,2.0
2,2001-04-15,2001-04-15 07:57:30,2001-04-15 08:15:30,FALSO,1.5
3,2001-04-15,2001-04-15 08:58:30,2001-04-15 09:07:30,FALSO,1.0
4,2001-04-15,2001-04-15 09:15:00,2001-04-15 09:21:00,FALSO,2.0
5,2001-04-15,2001-04-15 09:45:00,2001-04-15 09:48:00,FALSO,2.0
...,...,...,...,...,...
234,2017-09-06,2017-09-06 17:01:00,2017-09-06 17:16:00,FALSO,1.0
235,2017-09-10,2017-09-10 14:12:00,2017-09-10 14:42:00,FALSO,1.0
236,2017-09-10,2017-09-10 15:52:00,2017-09-10 16:30:00,VERDADERO,2.0


In [122]:
margin(combined)
combined[combined['resultat'] == 'VERDADERO']
grouped_df = combined.groupby('group 2')
final = pd.DataFrame()
final = grouped_df.max()
final['hora inici'] = grouped_df['hora inici'].min()
final['metode'] = grouped_df['metode'].mean()
final[final['resultat'] == 'VERDADERO']

,dia,hora inici,hora final,resultat,metode,diff,group 2
group,,,,,,,
10,2001-04-15,2001-04-15 13:38:30,2001-04-15 14:11:00,VERDADERO,1.500000,True,9
24,2001-04-26,2001-04-26 12:58:30,2001-04-26 13:24:00,VERDADERO,1.500000,True,23
30,2001-12-13,2001-12-13 14:19:30,2001-12-13 14:43:00,VERDADERO,1.500000,True,29
35,2004-07-16,2004-07-16 13:48:30,2004-07-16 14:37:00,VERDADERO,1.666667,True,34
49,2004-07-20,2004-07-20 12:22:30,2004-07-20 12:32:00,VERDADERO,1.500000,True,48
71,2004-08-14,2004-08-14 13:37:00,2004-08-14 13:51:00,VERDADERO,2.000000,True,69
75,2004-08-15,2004-08-15 12:34:30,2004-08-15 12:54:00,VERDADERO,1.500000,True,73
78,2005-05-27,2005-05-27 12:22:30,2005-05-27 12:29:00,VERDADERO,1.500000,True,76
80,2005-09-09,2005-09-09 09:50:30,2005-09-09 10:15:00,VERDADERO,1.500000,True,78
